# RF Complexity Analysis (EU)
## Data Loading

We use the `forest_report.json` file in the repo. Each entry contains the dataset name, metadata (sizes, series length, split), and trained forest statistics.

Il conteggio delle ragioni (`R`, `NR`, ecc.) viene estratto una sola volta: il risultato viene salvato in `results/redis_reason_counts.csv` e riutilizzato ai run successivi. Imposta `FORCE_RESULTS_REFRESH=1` (o passa `refresh=True` al loader) per rigenerare il file quando arrivano nuovi risultati.

In [1]:
from etl.tables import (
    prepare_models_analysis,
    print_models_analysis_overview,
    print_models_analysis_diagnostics,
    print_color_legend,
)

analysis_context = prepare_models_analysis(verbose=True)
print_models_analysis_overview(analysis_context)


Using cached redis summary (cached_at=2025-10-27T13:15:20.261007)
Base dir           : C:\Users\danie\Projects\GitHub\drifts
Loaded 88 rows from C:\Users\danie\Projects\GitHub\drifts\forest_report.json
Results directory  : C:\Users\danie\Projects\GitHub\drifts\results (exists=True)
Summary cache path : C:\Users\danie\Projects\GitHub\drifts\results\redis_reason_counts.csv
Using cached counts: True


In [2]:
analysis_context.first_table.summary_styler

,dataset,analyzed,train_size,test_size,series_length,n_estimators,mean eu features,eu std,eu_min,eu_max
0,Wine,YES,57,54,234,10,3.453,0.710,3.000,6.000
1,Wafer,YES,1000,6164,152,10,4.674,1.629,3.000,10.000
2,MiddlePhalanxOutlineCorrect,YES,600,291,80,10,18.000,5.725,7.000,39.000
3,MelbournePedestrian,YES,1138,2319,24,10,60.667,10.515,37.000,82.000
4,ChlorineConcentration,NO,467,3840,166,10,10.795,3.718,4.000,25.000
5,ScreenType,NO,375,375,720,10,4.363,1.421,3.000,10.000
6,FordA,NO,3601,1320,500,10,17.622,4.115,7.000,31.000
7,FordB,NO,3636,810,500,10,17.928,4.101,8.000,33.000
8,ElectricDevices,NO,8926,7711,96,10,310.906,61.024,189.000,433.000
9,SonyAIBORobotSurface1,YES,20,601,70,17,3.312,0.583,3.000,5.000


In [3]:
print_models_analysis_diagnostics(analysis_context)
analysis_context.summary_styler
print_color_legend()


? BASE_DIR     : C:\Users\danie\Projects\GitHub\drifts
? RESULTS_DIR  : C:\Users\danie\Projects\GitHub\drifts\results
? FOREST_REPORT: True
? SUMMARY_FILE : C:\Users\danie\Projects\GitHub\drifts\results\redis_reason_counts.csv
? USED_CACHE   : True
Tutti gli zip contengono redis_backup_db*.json (o non ci sono zip).
Dataset conteggiati: ECG200, HandOutlines, MelbournePedestrian, MiddlePhalanxOutlineCorrect, SonyAIBORobotSurface1, Wafer, Wine
COLUMN COLOR GRADIENTS
  n_estimators           -> Reds (min -> max)
  eu_complexity          -> Oranges (min -> max)
  series_length          -> YlOrBr (min -> max)
  n_features             -> YlGn (min -> max)
  mean_eu                -> Greens (min -> max)
  mean eu features       -> Greens (min -> max)
  eu_min                 -> Greens (min -> max)
  eu_max                 -> Greens (min -> max)
  eu_std_dev             -> Greens (min -> max)
  eu std                 -> Greens (min -> max)
  Candidate              -> Purples (min -> max)
  Reas

In [4]:
analysis_context.combined_analyzed_styler

dataset,Wine,Wafer,MiddlePhalanxOutlineCorrect,MelbournePedestrian,SonyAIBORobotSurface1,HandOutlines,ECG200
dataset,Wine,Wafer,MiddlePhalanxOutlineCorrect,MelbournePedestrian,SonyAIBORobotSurface1,HandOutlines,ECG200
train_size,57,1000,600,1138,20,1000,100
test_size,54,6164,291,2319,601,370,100
series_length,234,152,80,24,70,2709,96
n_estimators,10,10,10,10,17,59,101
mean eu features,3.453,4.674,18.000,60.667,3.312,3.252,4.042
eu std,0.710,1.629,5.725,10.515,0.583,0.558,1.148
total time,580.118,35463.843,1469.590,27536.924,93.066,5999.882,45937.485
ICF checks,2164,6174,976,7789,32738,5141,3507
Reason check iteration,235916,196066056,228277,223435165,37299,2458537,13825002
